In [1]:
import logging
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from prophet import Prophet
from sklearn.metrics import (
    mean_squared_error,
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
)
import yfinance

logging.getLogger("prophet").setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").disabled = True

Importing plotly failed. Interactive plots will not work.


# Model

In [2]:
def get_group_definition(X, pool_cols, pool_type):
    if pool_type == "complete":
        group = np.zeros(len(X), dtype="int")
        group_mapping = {0: "all"}
        n_groups = 1
    else:
        X[pool_cols] = pd.Categorical(X[pool_cols])
        group = X[pool_cols].cat.codes.values
        group_mapping = dict(enumerate(X[pool_cols].cat.categories))
        n_groups = X[pool_cols].nunique()
    return group, n_groups, group_mapping


class TimeSeriesModel:
    def _scale_data(self):
        self.y_min = 0
        self.y_max = self.data["y"].abs().max()
        self.ds_min = self.data["ds"].min()
        self.ds_max = self.data["ds"].max()

        self.data["y"] = self.data["y"] / self.y_max
        self.data["t"] = (self.data["ds"] - self.ds_min) / (self.ds_max - self.ds_min)

    def _process_data(self):
        self.data["ds"] = pd.to_datetime(self.data["ds"])
        self.data.sort_values("ds", inplace=True)
        self._scale_data()

    def _model_init(self):
        i0, i1 = self.data["ds"].idxmin(), self.data["ds"].idxmax()
        T = self.data["t"].iloc[i1] - self.data["t"].iloc[i0]
        slope = (self.data["y"].iloc[i1] - self.data["y"].iloc[i0]) / T
        intercept = self.data["y"].iloc[i0] - slope * self.data["t"].iloc[i0]
        return {
            "slope": slope,
            "intercept": intercept,
            "delta": 0.0,
            "beta": 0.0,
            "sigma": 1.0,
        }

    def fit(
        self,
        data,
        sigma_sd=0.5,
        mcmc_samples=0,
        chains=4,
        cores=4,
        use_prophet_initvals=True,
        progressbar=True,
    ):
        self.mcmc_samples = mcmc_samples

        self.data = data.reset_index(drop=True)
        self._process_data()

        self.initvals = {}
        # if use_prophet_initvals:
        #     self.initvals = self._model_init()

        self.model = pm.Model()
        self.model_idxs = {}
        mu = self.definition(self.model, self.data, self.initvals, self.model_idxs)

        with self.model:
            sigma = pm.HalfNormal(
                "sigma", sigma_sd, initval=self.initvals.get("sigma", 1)
            )
            _ = pm.Normal("obs", mu=mu, sigma=sigma, observed=self.data["y"])

            self.map_approx = None
            self.trace = None
            if self.mcmc_samples == 0:
                self.map_approx = pm.find_MAP(progressbar=progressbar, maxeval=1e4)
            else:
                self.trace = pm.sample(self.mcmc_samples, chains=chains, cores=cores)

    def tune(
        self,
        data,
        sigma_sd=0.5,
        mcmc_samples=0,
        chains=4,
        cores=4,
        use_prophet_initvals=True,
        progressbar=True,
    ):
        self.mcmc_samples = mcmc_samples

        self.data = data.reset_index(drop=True)
        self._process_data()

        self.initvals = {}
        # if use_prophet_initvals:
        #     self.initvals = self._model_init()

        self.model = pm.Model()
        self.model_idxs = {}
        # model, data, initvals, model_idxs, prev
        mu = self._tune(
            self.model, self.data, self.initvals, self.model_idxs, self.map_approx
        )

        with self.model:
            sigma = pm.HalfNormal(
                "sigma", sigma_sd, initval=self.initvals.get("sigma", 1)
            )
            _ = pm.Normal("obs", mu=mu, sigma=sigma, observed=self.data["y"])

            self.map_approx = None
            self.trace = None
            if self.mcmc_samples == 0:
                self.map_approx = pm.find_MAP(progressbar=progressbar, maxeval=1e4)
            else:
                self.trace = pm.sample(self.mcmc_samples, chains=chains, cores=cores)

    def _make_future_df(self, days):
        future = pd.DataFrame(
            {
                "ds": pd.DatetimeIndex(
                    np.hstack(
                        (
                            self.data["ds"].unique().to_numpy(),
                            pd.date_range(
                                self.ds_max,
                                self.ds_max + pd.Timedelta(days, "D"),
                                inclusive="right",
                            ).to_numpy(),
                        )
                    )
                )
            }
        )
        future["t"] = (future["ds"] - self.ds_min) / (self.ds_max - self.ds_min)
        return future

    def predict(self, days):
        future = self._make_future_df(days)
        forecasts = self._predict(
            future, self.mcmc_samples, self.map_approx, self.trace
        )

        for group_code in range(forecasts.shape[0]):
            future[f"yhat_{group_code}"] = forecasts[group_code] * self.y_max
            for model_type, model_cnt in self.model_idxs.items():
                if model_type.startswith("fs"):
                    continue
                for model_idx in range(model_cnt):
                    component = f"{model_type}_{model_idx}_{group_code}"
                    if component in future.columns:
                        future[component] *= self.y_max

        return future

    def _predict(self, future, mcmc_samples, map_approx, trace):
        if mcmc_samples == 0:
            return self._predict_map(future, map_approx)

        return self._predict_mcmc(future, trace)

    def plot(self, future, y_true=None, pool_cols=None):
        plt.figure(figsize=(14, 100 * 6))
        plt.subplot(100, 1, 1)
        plt.title("Predictions")
        plt.grid()

        group, _, groups_ = get_group_definition(self.data, pool_cols, "not_complete")
        for group_code, group_name in groups_.items():
            group_idx = group == group_code
            color = np.random.rand(3)
            plt.scatter(
                self.data["ds"][group_idx],
                self.data["y"][group_idx] * self.y_max,
                s=0.5,
                color=color,
                label=group_name,
            )

        if y_true is not None:
            test_group, _, test_groups_ = get_group_definition(
                y_true, pool_cols, "not_complete"
            )
            for group_code, group_name in test_groups_.items():
                group_idx = test_group == group_code
                color = np.random.rand(3)
                plt.scatter(
                    y_true["ds"][group_idx],
                    y_true["y"][group_idx],
                    s=0.5,
                    color=color,
                    label=f"y - {group_name}",
                )

        for group_code, group_name in groups_.items():
            plt.plot(
                future["ds"],
                future[f"yhat_{group_code}"],
                lw=1,
                label=f"yhat - {group_name}",
            )

        plt.legend()
        plot_params = {"idx": 1}
        self._plot(plot_params, future, self.data, self.y_max, y_true)

    def metrics(self, y_true, future, pool_cols=None, pool_type="individual"):
        metrics = {"mse": {}, "rmse": {}, "mae": {}, "mape": {}}
        test_group, _, test_groups_ = get_group_definition(y_true, pool_cols, pool_type)
        for group_code, group_name in test_groups_.items():
            group_idx = test_group == group_code
            y = y_true["y"][group_idx]
            yhat = future[f"yhat_{group_code}"][-len(y) :]
            metrics["mse"][group_name] = mean_squared_error(y, yhat)
            metrics["rmse"][group_name] = root_mean_squared_error(y, yhat)
            metrics["mae"][group_name] = mean_absolute_error(y, yhat)
            metrics["mape"][group_name] = mean_absolute_percentage_error(y, yhat)

        return pd.DataFrame(metrics)

    def __add__(self, other):
        return AdditiveTimeSeries(self, other)

    def __mul__(self, other):
        return MultiplicativeTimeSeries(self, other)


class AdditiveTimeSeries(TimeSeriesModel):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def definition(self, *args, **kwargs):
        return self.left.definition(*args, **kwargs) + self.right.definition(
            *args, **kwargs
        )

    def _tune(self, *args, **kwargs):
        return self.left._tune(*args, **kwargs) + self.right._tune(*args, **kwargs)

    def _predict(self, *args, **kwargs):
        return self.left._predict(*args, **kwargs) + self.right._predict(
            *args, **kwargs
        )

    def _plot(self, *args, **kwargs):
        self.left._plot(*args, **kwargs)
        self.right._plot(*args, **kwargs)

    def __str__(self):
        return f"{self.left} + {self.right}"


class MultiplicativeTimeSeries(TimeSeriesModel):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def definition(self, *args, **kwargs):
        return self.left.definition(*args, **kwargs) * (
            1 + self.right.definition(*args, **kwargs)
        )

    def _tune(self, *args, **kwargs):
        return self.left._tune(*args, **kwargs) * (
            1 + self.right._tune(*args, **kwargs)
        )

    def _predict(self, *args, **kwargs):
        return self.left._predict(*args, **kwargs) * (
            1 + self.right._predict(*args, **kwargs)
        )

    def _plot(self, *args, **kwargs):
        self.left._plot(*args, **kwargs)
        self.right._plot(*args, **kwargs)

    def __str__(self):
        left = f"{self.left}"
        if type(self.left) is AdditiveTimeSeries:
            left = f"({self.left})"

        right = f"{self.right}"
        if type(self.right) is AdditiveTimeSeries:
            right = f"({self.right})"

        return f"{left} * {right}"


class LinearTrend(TimeSeriesModel):
    def __init__(
        self,
        n_changepoints=25,
        changepoint_range=0.8,
        slope_mean=0,
        slope_sd=5,
        intercept_mean=0,
        intercept_sd=5,
        delta_mean=0,
        delta_sd=0.05,
        pool_cols=None,
        pool_type="complete",
        allow_tune=False,
    ):
        self.n_changepoints = n_changepoints
        self.changepoint_range = changepoint_range
        self.slope_mean = slope_mean
        self.slope_sd = slope_sd
        self.intercept_mean = intercept_mean
        self.intercept_sd = intercept_sd
        self.delta_mean = delta_mean
        self.delta_sd = delta_sd

        self.pool_cols = pool_cols
        self.pool_type = pool_type
        self.allow_tune = allow_tune

    def definition(self, model, data, initvals, model_idxs):
        model_idxs["lt"] = model_idxs.get("lt", 0)
        self.model_idx = model_idxs["lt"]
        model_idxs["lt"] += 1

        self.group, self.n_groups, self.groups_ = get_group_definition(
            data, self.pool_cols, self.pool_type
        )

        with model:
            if self.pool_type == "partial":
                sigma_slope = pm.HalfCauchy(
                    f"lt_{self.model_idx} - sigma_slope", beta=self.slope_sd
                )
                offset_slope = pm.Normal(
                    f"lt_{self.model_idx} - offset_slope",
                    mu=0,
                    sigma=1,
                    shape=self.n_groups,
                )
                slope = pm.Deterministic(
                    f"lt_{self.model_idx} - slope", offset_slope * sigma_slope
                )

                delta_sd = self.delta_sd
                if self.delta_sd is None:
                    delta_sd = pm.Exponential(f"lt_{self.model_idx} - tau", 1.5)

                sigma_delta = pm.HalfCauchy(
                    f"lt_{self.model_idx} - sigma_delta", beta=delta_sd
                )
                offset_delta = pm.Laplace(
                    f"lt_{self.model_idx} - offset_delta",
                    0,
                    1,
                    shape=(self.n_groups, self.n_changepoints),
                )
                delta = pm.Deterministic(
                    f"lt_{self.model_idx} - delta", offset_delta * sigma_delta
                )
            else:
                slope = pm.Normal(
                    f"lt_{self.model_idx} - slope",
                    self.slope_mean,
                    self.slope_sd,
                    initval=initvals.get("slope", None),
                    shape=self.n_groups,
                )

                delta_sd = self.delta_sd
                if self.delta_sd is None:
                    delta_sd = pm.Exponential(f"lt_{self.model_idx} - tau", 1.5)

                delta = pm.Laplace(
                    f"lt_{self.model_idx} - delta",
                    self.delta_mean,
                    delta_sd,
                    shape=(self.n_groups, self.n_changepoints),
                )

            intercept = pm.Normal(
                f"lt_{self.model_idx} - intercept",
                self.intercept_mean,
                self.intercept_sd,
                initval=initvals.get("intercept", None),
                shape=self.n_groups,
            )

            if self.pool_type == "individual":
                ss = []
                t = np.array(data["t"])
                for group_code in range(self.n_groups):
                    series_data = data[self.group == group_code]
                    hist_size = int(
                        np.floor(series_data.shape[0] * self.changepoint_range)
                    )
                    cp_indexes = (
                        np.linspace(0, hist_size - 1, self.n_changepoints + 1)
                        .round()
                        .astype(int)
                    )
                    ss.append(np.array(series_data.iloc[cp_indexes]["t"].tail(-1)))

                self.s = np.stack(ss, axis=0)
                A = (t[:, None] > self.s[self.group]) * 1

                gamma = -self.s[self.group, :] * delta[self.group, :]
                trend = pm.Deterministic(
                    f"lt_{self.model_idx} - trend",
                    (slope[self.group] + pm.math.sum(A * delta[self.group], axis=1)) * t
                    + (intercept[self.group] + pm.math.sum(A * gamma, axis=1)),
                )
            else:
                t = np.array(data["t"])
                hist_size = int(np.floor(data.shape[0] * self.changepoint_range))
                cp_indexes = (
                    np.linspace(0, hist_size - 1, self.n_changepoints + 1)
                    .round()
                    .astype(int)
                )
                self.s = np.array(data.iloc[cp_indexes]["t"].tail(-1))
                A = (t[:, None] > self.s) * 1

                gamma = -self.s * delta[self.group, :]
                trend = pm.Deterministic(
                    f"lt_{self.model_idx} - trend",
                    (slope[self.group] + pm.math.sum(A * delta[self.group], axis=1)) * t
                    + (intercept[self.group] + pm.math.sum(A * gamma, axis=1)),
                )

        return trend

    def _tune(self, model, data, initvals, model_idxs, prev):
        return self.definition(model, data, initvals, model_idxs)

    def _predict_map(self, future, map_approx):
        forecasts = []
        if self.pool_type != "individual":
            new_A = (np.array(future["t"])[:, None] > self.s) * 1

        for group_code in self.groups_.keys():
            if self.pool_type == "individual":
                s = self.s[group_code]
                new_A = (np.array(future["t"])[:, None] > self.s[group_code]) * 1
            else:
                s = self.s

            forecasts.append(
                np.array(
                    (
                        map_approx[f"lt_{self.model_idx} - slope"][group_code]
                        + np.dot(
                            new_A,
                            map_approx[f"lt_{self.model_idx} - delta"][group_code],
                        )
                    )
                    * future["t"]
                    + (
                        map_approx[f"lt_{self.model_idx} - intercept"][group_code]
                        + np.dot(
                            new_A,
                            (
                                -s
                                * map_approx[f"lt_{self.model_idx} - delta"][group_code]
                            ),
                        )
                    )
                )
            )
            future[f"lt_{self.model_idx}_{group_code}"] = forecasts[-1]

        return np.vstack(forecasts)

    def _predict_mcmc(self, future, trace):
        forecasts = []
        if self.pool_type == "individual":
            new_A = (np.array(future["t"])[:, None] > self.s[self.group]) * 1
        else:
            new_A = (np.array(future["t"])[:, None] > self.s) * 1

        for group_code in self.groups_.keys():
            delta = (
                trace["posterior"][f"lt_{self.model_idx} - delta"]
                .to_numpy()[:, :, group_code]
                .mean(0)
            )
            slope = (
                trace["posterior"][f"lt_{self.model_idx} - slope"]
                .to_numpy()[:, :, group_code]
                .mean(0)
            )
            intercept = (
                trace["posterior"][f"lt_{self.model_idx} - intercept"]
                .to_numpy()[:, :, group_code]
                .mean(0)
            )

            forecasts.append(
                (
                    (slope + np.dot(new_A, delta.T)).T * future["t"].to_numpy()
                    + (intercept + np.dot(new_A, (-self.s * delta).T)).T
                ).mean(0)
            )
            future[f"lt_{self.model_idx}_{group_code}"] = forecasts[-1]

        return np.vstack(forecasts)

    def _plot(self, plot_params, future, data, y_max, y_true=None):
        plot_params["idx"] += 1
        plt.subplot(100, 1, plot_params["idx"])
        plt.title(f"lt_{self.model_idx}")
        plt.grid()

        for group_code, group_name in self.groups_.items():
            plt.plot(
                future["ds"],
                future[f"lt_{self.model_idx}_{group_code}"],
                lw=1,
                label=group_name,
            )

        plt.legend()

    def __str__(self):
        return (
            f"LT(n={self.n_changepoints},r={self.changepoint_range},at={self.allow_tune},{self.pool_type})"
        )


class FourierSeasonality(TimeSeriesModel):
    def __init__(
        self,
        period,
        series_order,
        beta_mean=0,
        beta_sd=10,
        shrinkage_strength=100,
        pool_cols=None,
        pool_type="complete",
        allow_tune=False,
    ):
        self.period = period
        self.series_order = series_order
        self.beta_mean = beta_mean
        self.beta_sd = beta_sd
        self.shrinkage_strength = shrinkage_strength

        self.pool_cols = pool_cols
        self.pool_type = pool_type
        self.allow_tune = allow_tune

    def _fourier_series(self, data):
        # convert to days since epoch
        NANOSECONDS_TO_SECONDS = 1000 * 1000 * 1000
        t = (
            data["ds"].to_numpy(dtype=np.int64)
            // NANOSECONDS_TO_SECONDS
            / (3600 * 24.0)
        )

        x_T = t * np.pi * 2
        fourier_components = np.empty((data["ds"].shape[0], 2 * self.series_order))
        for i in range(self.series_order):
            c = x_T * (i + 1) / self.period
            fourier_components[:, 2 * i] = np.sin(c)
            fourier_components[:, (2 * i) + 1] = np.cos(c)

        return fourier_components

    def definition(self, model, data, initvals, model_idxs):
        model_idxs["fs"] = model_idxs.get("fs", 0)
        self.model_idx = model_idxs["fs"]
        model_idxs["fs"] += 1

        group, n_groups, self.groups_ = get_group_definition(
            data, self.pool_cols, self.pool_type
        )

        x = self._fourier_series(data)
        beta_initval = initvals.get("beta", None)
        if beta_initval is not None:
            beta_initval = np.array([beta_initval] * 2 * self.series_order)

        with model:
            if self.pool_type == "partial":
                # shift_t = pm.Uniform(
                #     f"fs_{self.model_idx} - shift_t(p={self.period},n={self.series_order})",
                #     lower=0,
                #     upper=self.period,
                #     shape=n_groups,
                # )
                mu_beta = pm.Normal(
                    f"fs_{self.model_idx} - beta_mu(p={self.period},n={self.series_order})",
                    mu=self.beta_mean,
                    sigma=self.beta_sd,
                    shape=2 * self.series_order,
                    initval=beta_initval,
                )
                sigma_beta = pm.HalfNormal(
                    f"fs_{self.model_idx} - beta_sigma(p={self.period},n={self.series_order})",
                    sigma=self.beta_sd / self.shrinkage_strength,
                    shape=2 * self.series_order,
                )
                offset_beta = pm.Normal(
                    f"fs_{self.model_idx} - offset_beta(p={self.period},n={self.series_order})",
                    mu=0,
                    sigma=1,
                    shape=(n_groups, 2 * self.series_order),
                )

                beta = pm.Deterministic(
                    f"fs_{self.model_idx} - beta(p={self.period},n={self.series_order})",
                    mu_beta + offset_beta * sigma_beta,
                )
            else:
                beta = pm.Normal(
                    f"fs_{self.model_idx} - beta(p={self.period},n={self.series_order})",
                    mu=self.beta_mean,
                    sigma=self.beta_sd,
                    shape=(n_groups, 2 * self.series_order),
                    initval=beta_initval,
                )

        return pm.math.sum(x * beta[group], axis=1)

    def _tune(self, model, data, initvals, model_idxs, prev):
        if not self.allow_tune:
            return self.definition(model, data, initvals, model_idxs)

        model_idxs["fs"] = model_idxs.get("fs", 0)
        self.model_idx = model_idxs["fs"]
        model_idxs["fs"] += 1

        group, n_groups, self.groups_ = get_group_definition(
            data, self.pool_cols, self.pool_type
        )

        x = self._fourier_series(data)
        beta_initval = initvals.get("beta", None)
        if beta_initval is not None:
            beta_initval = np.array([beta_initval] * 2 * self.series_order)

        with model:
            sigma_beta = pm.HalfNormal(
                f"fs_{self.model_idx} - beta_sigma(p={self.period},n={self.series_order})",
                sigma=self.beta_sd / self.shrinkage_strength,
                shape=2 * self.series_order,
            )
            offset_beta = pm.Normal(
                f"fs_{self.model_idx} - offset_beta(p={self.period},n={self.series_order})",
                mu=0,
                sigma=1,
                shape=(n_groups, 2 * self.series_order),
            )

            beta = pm.Deterministic(
                f"fs_{self.model_idx} - beta(p={self.period},n={self.series_order})",
                prev[
                    f"fs_{self.model_idx} - beta(p={self.period},n={self.series_order})"
                ]
                + offset_beta * sigma_beta,
            )

        return pm.math.sum(x * beta[group], axis=1)

    def _det_seasonality_posterior(self, beta, x):
        return np.dot(x, beta.T)

    def _predict_map(self, future, map_approx):
        forecasts = []
        for group_code in self.groups_.keys():
            forecasts.append(
                self._det_seasonality_posterior(
                    map_approx[
                        f"fs_{self.model_idx} - beta(p={self.period},n={self.series_order})"
                    ][group_code],
                    self._fourier_series(future),
                )
            )
            future[f"fs_{self.model_idx}_{group_code}"] = forecasts[-1]

        return np.vstack(forecasts)

    def _predict_mcmc(self, future, trace):
        forecasts = []
        for group_code in self.groups_.keys():
            forecasts.append(
                self._det_seasonality_posterior(
                    trace["posterior"][
                        f"fs_{self.model_idx} - beta(p={self.period},n={self.series_order})"
                    ]
                    .to_numpy()[:, :, group_code]
                    .mean(0),
                    self._fourier_series(future),
                ).T.mean(0)
            )
            future[f"fs_{self.model_idx}_{group_code}"] = forecasts[-1]

        return np.vstack(forecasts)

    def _plot(self, plot_params, future, data, y_max, y_true=None):
        date = future["ds"] if self.period > 7 else future["ds"].dt.day_name()
        plot_params["idx"] += 1
        plt.subplot(100, 1, plot_params["idx"])
        plt.title(f"fs_{self.model_idx} - p={self.period},n={self.series_order}")
        plt.grid()

        for group_code, group_name in self.groups_.items():
            plt.plot(
                date[-int(self.period) :],
                future[f"fs_{self.model_idx}_{group_code}"][-int(self.period) :],
                lw=1,
                label=group_name,
            )

        plt.legend()

    def __str__(self):
        return f"FS(p={self.period},n={self.series_order},at={self.allow_tune},{self.pool_type})"


class Constant(TimeSeriesModel):
    def __init__(self, lower, upper, pool_cols=None, pool_type="complete", allow_tune=False):
        self.lower = lower
        self.upper = upper

        self.pool_cols = pool_cols
        self.pool_type = pool_type
        self.allow_tune = allow_tune

    def definition(self, model, data, initvals, model_idxs):
        model_idxs["c"] = model_idxs.get("c", 0)
        self.model_idx = model_idxs["c"]
        model_idxs["c"] += 1

        group, n_groups, self.groups_ = get_group_definition(
            data, self.pool_cols, self.pool_type
        )

        with model:
            if self.pool_type == "partial":
                mu_c = pm.Uniform(
                    f"c_{self.model_idx} - mu_c(l={self.lower},u={self.upper})",
                    lower=self.lower,
                    upper=self.upper,
                    shape=n_groups,
                )
                offset_c = pm.Normal(
                    f"c_{self.model_idx} - offset_c(l={self.lower},u={self.upper})",
                    mu=0,
                    sigma=1,
                    shape=n_groups,
                )
                c = pm.Deterministic(
                    f"c_{self.model_idx} - c(l={self.lower},u={self.upper})",
                    mu_c + offset_c,
                )
            else:
                c = pm.Uniform(
                    f"c_{self.model_idx} - c(l={self.lower},u={self.upper})",
                    lower=self.lower,
                    upper=self.upper,
                    shape=n_groups,
                )

        return c[group]
    
    def _tune(self, model, data, initvals, model_idxs, prev):
        return self.definition(model, data, initvals, model_idxs)

    def _predict_map(self, future, map_approx):
        forecasts = []
        for group_code in self.groups_.keys():
            forecasts.append(
                np.ones_like(future["t"])
                * map_approx[f"c_{self.model_idx} - c(l={self.lower},u={self.upper})"][
                    group_code
                ]
            )
            future[f"c_{self.model_idx}_{group_code}"] = forecasts[-1]

        return np.vstack(forecasts)

    def _predict_mcmc(self, future, trace):
        forecasts = []
        for group_code in self.groups_.keys():
            forecasts.append(
                np.ones_like(future["t"])
                * trace["posterior"][
                    f"c_{self.model_idx} - c(l={self.lower},u={self.upper})"
                ]
                .to_numpy()[:, :, group_code]
                .mean()
            )
            future[f"c_{self.model_idx}_{group_code}"] = forecasts[-1]

        return np.vstack(forecasts)

    def _plot(self, plot_params, future, data, y_max, y_true=None):
        plot_params["idx"] += 1
        plt.subplot(100, 1, plot_params["idx"])
        plt.title(f"c_{self.model_idx} - c(l={self.lower},u={self.upper})")

        plot_data = []
        for group_code, group_name in self.groups_.items():
            plot_data.append(
                (group_name, future[f"c_{self.model_idx}_{group_code}"][0])
            )

        plt.bar(*zip(*plot_data))
        plt.axhline(0, c="k", linewidth=3)

    def __str__(self):
        return f"C(l={self.lower},u={self.upper},at={self.allow_tune},{self.pool_type})"

# Data fetcher

In [3]:
indexes = ["^W5000", "^GSPC", "^IXIC", "^DJI"]

gspc_tickers = [
    "AAPL", "MSFT", "AMZN", "FB", "TSLA", "GOOGL", "GOOG", "JNJ", "JPM", "V",
    "PG", "UNH", "DIS", "NVDA", "MA", "HD", "PYPL", "VZ", "ADBE", "CMCSA",
    "NFLX", "BAC", "KO", "MRK", "PEP", "T", "PFE", "INTC", "CRM", "WMT", "ABT",
    "ABBV", "CSCO", "TMO", "NKE", "AVGO", "XOM", "QCOM", "COST", "ACN", "CVX",
    "MCD", "MDT", "NEE", "TXN", "HON", "DHR", "UNP", "BMY", "LIN", "LLY",
    "AMGN", "PM", "C", "SBUX", "WFC", "ORCL", "UPS", "LOW", "BA", "IBM", "AMD",
    "RTX", "NOW", "BLK", "MMM", "INTU", "AMT", "CAT", "MS", "CHTR", "ISRG",
    "GE", "BKNG", "GS", "CVS", "TGT", "FIS", "LMT", "DE", "MU", "MDLZ", "TJX",
    "SYK", "ANTM", "SCHW", "SPGI", "AXP", "AMAT", "TMUS", "ZTS", "MO", "ADP",
    "CI", "PLD", "CL", "GILD", "BDX", "ATVI", "CB", "CSX", "CCI", "LRCX",
    "DUK", "ADSK", "FISV", "CME", "SO", "ICE", "TFC", "GPN", "USB", "EQIX",
    "PNC", "FDX", "VRTX", "D", "APD", "NSC", "EL", "SHW", "MMC", "ITW", "PGR",
    "EW", "ADI", "HUM", "ILMN", "ECL", "GM", "DD", "DG", "BSX", "REGN", "AON",
    "NEM", "EMR", "ETN", "NOC", "MCO", "KMB", "WM", "COF", "ROP", "CTSH",
    "ROST", "HCA", "TWTR", "COP", "IDXX", "EA", "AEP", "EXC", "DOW", "BAX",
    "TEL", "KLAC", "LHX", "SNPS", "APH", "DLR", "CMG", "ALGN", "CDNS", "SYY",
    "FCX", "BIIB", "STZ", "MSCI", "SRE", "A", "MCHP", "GIS", "MET", "TRV",
    "DXCM", "APTV", "PSA", "PH", "MAR", "XEL", "TT", "CNC", "XLNX", "GD", "BK",
    "F", "IQV", "TROW", "ALXN", "MNST", "PPG", "HPQ", "VRSK", "JCI", "TDG",
    "CMI", "INFO", "ALL", "EBAY", "ORLY", "YUM", "AIG", "ZBH", "SBAC", "ANSS",
    "CTAS", "PRU", "HLT", "RMD", "CARR", "PSX", "BLL", "SLB", "PCAR", "PAYX",
    "ES", "PEG", "ROK", "EOG", "AFL", "WEC", "CTVA", "MSI", "WBA", "SWK",
    "ADM", "FAST", "SPG", "MCK", "AME", "AWK", "DFS", "LUV", "OTIS", "GLW",
    "AZO", "VFC", "WLTW", "MTD", "WELL", "MPC", "KMI", "CPRT", "STT", "DAL",
    "FRC", "CLX", "DLTR", "SWKS", "WY", "ED", "KR", "KEYS", "WMB", "CERN",
    "TTWO", "FTV", "AJG", "EIX", "MKC", "MXIM", "LYB", "DTE", "EFX", "VLO",
    "BBY", "AMP", "DHI", "FLT", "VTRS", "HSY", "KHC", "AVB", "PAYC", "ETSY",
    "O", "VRSN", "PPL", "CHD", "MKTX", "ARE", "VIAC", "CBRE", "LEN", "WST",
    "ZBRA", "EQR", "RSG", "SIVB", "FTNT", "ETR", "TER", "LH", "VMC", "FITB",
    "LVS", "IP", "NTRS", "AEE", "TFX", "KSU", "QRVO", "TSN", "SYF", "CDW",
    "ODFL", "PXD", "HOLX", "AMCR", "GWW", "VTR", "XYL", "DOV", "EXPE", "GRMN",
    "COO", "CAG", "BR", "MLM", "TYL", "HIG", "CMS", "CTLT", "AKAM", "OKE",
    "IR", "WDC", "URI", "HAL", "FE", "TSCO", "MTB", "PEAK", "INCY", "ULTA",
    "STE", "CCL", "EXPD", "PKI", "NUE", "DGX", "KEY", "CTXS", "VAR", "K",
    "ANET", "CAH", "ALB", "AES", "DRI", "KMX", "RF", "ESS", "WAT", "CFG",
    "HPE", "NDAQ", "CE", "DPZ", "IEX", "EXR", "POOL", "FMC", "DRE", "NTAP",
    "ABMD", "OXY", "MAA", "GPC", "TDY", "HES", "ABC", "MAS", "IT", "NVR",
    "TIF", "J", "LDOS", "BKR", "STX", "RCL", "EMN", "OMC", "BXP", "SJM", "WAB",
    "HRL", "PKG", "CINF", "AVY", "MGM", "LNT", "HBAN", "CHRW", "PFG", "UAL",
    "EVRG", "BIO", "JKHY", "NLOK", "HAS", "ATO", "FBHS", "CNP", "RJF", "IFF",
    "PHM", "LW", "CXO", "XRAY", "WRK", "JBHT", "UDR", "WHR", "HWM", "TXT",
    "WYNN", "FFIV", "ALLE", "AAP", "UHS", "L", "LYV", "HST", "CBOE", "PWR",
    "LKQ", "FOXA", "CPB", "AAL", "LUMN", "HSIC", "BWA", "RE", "WRB", "SNA",
    "IPG", "NRG", "GL", "LNC", "WU", "PNW", "PNR", "NI", "LB", "DVA", "ROL",
    "TPR", "TAP", "IRM", "MHK", "CF", "AIZ", "NCLH", "NWL", "DISH", "IPGP",
    "MOS", "CMA", "DISCK", "FANG", "NLSN", "AOS", "JNPR", "REG", "ZION", "RHI",
    "SEE", "NWSA", "HII", "BEN", "PVH", "IVZ", "DXC", "COG", "KIM", "ALK",
    "PRGO", "DVN", "LEG", "FRT", "VNO", "FLIR", "PBCT", "APA", "NOV", "MRO",
    "HBI", "RL", "DISCA", "FLS", "UNM", "VNT", "FOX", "SLG", "GPS", "FTI",
    "XRX", "HFC", "UAA", "UA", "NWS"
]

dji_tickers = [
    "DIS", "WMT", "DOW", "NKE", "CRM", "HD", "V", "MSFT", "MMM", "CSCO", "KO",
    "AAPL", "HON", "JNJ", "TRV", "PG", "CVX", "VZ", "CAT", "BA", "AMGN", "IBM",
    "AXP", "JPM", "WBA", "MCD", "MRK", "GS", "UNH", "INTC"
]

ixic_tickers = [
    "FEYE", "ATEC", "SLAB", "CMRX", "NVCR", "FNLC", "NMRK", "SCOR", "AGLE",
    "FARO", "OLMA", "TSLA", "FRTA", "AKTX", "KLXE", "CVCO", "NVCN", "EXAS",
    "SDC", "BBQ", "IFRX", "CIIC", "BBI", "FNKO", "TWST", "FARM", "ACCD",
    "NMRD", "FRSX", "OPTT"
]

In [4]:
def fetch_data(tickers, start="1970-01-01", end="2020-01-01"):
    data = yfinance.download(
        tickers,
        interval="1d",
        start=start,
        end=end,
    )
    downloaded_tickers = {col[1] for col in data.columns}
    dfs = []
    for ticker in downloaded_tickers:
        df = pd.DataFrame(
            data={
                "open": data["Open"][ticker].to_numpy(),
                "high": data["High"][ticker].to_numpy(),
                "low": data["Low"][ticker].to_numpy(),
                "close": data["Close"][ticker].to_numpy(),
                "typical_price": (
                    (
                        data["Open"][ticker]
                        + data["High"][ticker]
                        + data["Low"][ticker]
                        + data["Close"][ticker]
                    )
                    / 4
                ).to_numpy(),
                "volume": data["Volume"][ticker].to_numpy(),
            },
            index=data["Close"][ticker].index,
        )

        full_date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
        df = df.reindex(full_date_range).interpolate()
        df["ds"] = df.index
        df.reset_index(drop=True, inplace=True)
        df["series"] = ticker
        dfs.append(df)

    return dfs

In [5]:
def generate_train_test_df(
    start,
    window,
    horizon,
    dfs,
    for_prophet=False,
    y_col="typical_price",
    perform_scaling=True,
):
    train_dfs = []
    test_dfs = []
    scales = []
    for df in dfs:
        train_df = df[start : start + window].copy()
        test_df = df[start + window : start + window + horizon].copy()
        if train_df.isna().any().any() or test_df.isna().any().any():
            continue

        train_df["y"] = train_df[y_col]
        test_df["y"] = test_df[y_col]

        if perform_scaling:
            scales.append(train_df[y_col].max())
            train_df["y"] = train_df[y_col] / scales[-1]
            test_df["y"] = test_df[y_col] / scales[-1]

        train_dfs.append(train_df)
        test_dfs.append(test_df)

    if len(train_dfs) == 0:
        return None

    if for_prophet:
        return train_dfs, test_dfs, scales

    return pd.concat(train_dfs), pd.concat(test_dfs), scales

In [6]:
def generate_train_test_df_around_point(
    window,
    horizon,
    dfs,
    point="2009-09-01",
    for_prophet=False,
    y_col="typical_price",
    perform_scaling=True,
):
    train_dfs = []
    test_dfs = []
    scales = []

    for df in dfs:
        point_idx = len(df[df["ds"] < point])
        check = generate_train_test_df(
            start=point_idx - window,
            window=window,
            horizon=horizon,
            dfs=[df],
            for_prophet=for_prophet,
            y_col=y_col,
            perform_scaling=perform_scaling,
        )
        if check is None:
            continue

        train_df, test_df, scale = check

        scales += scale

        if for_prophet:
            train_dfs += train_df
            test_dfs += test_df
        else:
            train_dfs.append(train_df)
            test_dfs.append(test_df)

    if len(train_dfs) == 0:
        return None
    
    if for_prophet:
        return train_dfs, test_dfs, scales
    
    return pd.concat(train_dfs), pd.concat(test_dfs), scales

# Generate models

In [7]:
# model_components = [
#     [LinearTrend(pool_cols="series", pool_type=pt) for pt in ["individual", "partial"]],
#     [
#         FourierSeasonality(
#             period=365.25, series_order=10, pool_cols="series", pool_type=pt
#         )
#         for pt in ["individual", "partial"]
#     ],
#     [
#         FourierSeasonality(
#             period=91.3125, series_order=n, pool_cols="series", pool_type=pt
#         )
#         for n in range(7, 10)
#         for pt in ["individual", "partial"]
#     ],
#     [
#         FourierSeasonality(
#             period=30.4375, series_order=n, pool_cols="series", pool_type=pt
#         )
#         for n in range(4, 7)
#         for pt in ["individual", "partial"]
#     ],
#     [
#         FourierSeasonality(period=7, series_order=3, pool_cols="series", pool_type=pt)
#         for pt in ["individual", "partial"]
#     ],
# ]

model_components = [
    [LinearTrend(allow_tune=False)],
    [
        FourierSeasonality(period=365.25, series_order=10, allow_tune=allow_tune)
        for allow_tune in [True, False]
    ],
    [
        FourierSeasonality(period=91.3125, series_order=n, allow_tune=allow_tune)
        for n in range(7, 10)
        for allow_tune in [True, False]
    ],
    [
        FourierSeasonality(period=30.4375, series_order=n, allow_tune=allow_tune)
        for n in range(4, 7)
        for allow_tune in [True, False]
    ],
    [
        FourierSeasonality(period=7, series_order=3, allow_tune=allow_tune)
        for allow_tune in [True, False]
    ],
]

In [8]:
q = [(0, [mc]) for mc in model_components[0]]
models = []

while len(q):
    level, model = q.pop(0)
    if level + 1 == len(model_components):
        models.append(model)
        continue

    mcs = model_components[level + 1]
    for mc in mcs:
        # if mc.pool_type == "partial":
        #     q.append(
        #         (
        #             level + 1,
        #             model
        #             + [
        #                 Constant(
        #                     lower=-1, upper=1, pool_cols="series", pool_type="partial"
        #                 )
        #                 * mc
        #             ],
        #         )
        #     )

        q.append((level + 1, model + [mc]))
        q.append((level + 1, model))

In [9]:
def sum_models(models):
    s = None
    for model in models:
        if s is None:
            s = model
        else:
            s += model

    return s

In [10]:
models = [model[0] * sum_models(model[1:]) if len(model) > 1 else model[0] for model in models]

In [11]:
str_models = {""}
final_models = []
for model in models:
    str_model = str(model)
    if str_model in str_models:
        continue

    str_models.add(str_model)
    final_models.append(model)

len(final_models)

441

# Test prophet

In [12]:
def get_prophet_metrics(y_trues, y_preds, horizon):
    result = None
    for y_true, y_pred in zip(y_trues, y_preds):
        group_name = y_true["series"].iloc[0]
        single_metrics = {"mse": {}, "rmse": {}, "mae": {}, "mape": {}}
        single_metrics["mse"][group_name] = mean_squared_error(
            y_true["y"], y_pred["yhat"][-horizon:]
        )
        single_metrics["rmse"][group_name] = root_mean_squared_error(
            y_true["y"], y_pred["yhat"][-horizon:]
        )
        single_metrics["mae"][group_name] = mean_absolute_error(
            y_true["y"], y_pred["yhat"][-horizon:]
        )
        single_metrics["mape"][group_name] = mean_absolute_percentage_error(
            y_true["y"], y_pred["yhat"][-horizon:]
        )
        if result is None:
            result = pd.DataFrame(single_metrics)
        else:
            result = pd.concat((result, pd.DataFrame(single_metrics)))

    return result

In [13]:
smp = fetch_data(["^GSPC"])
smp_tickers = fetch_data(gspc_tickers, start="2000-01-01", end="2011-01-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  503 of 503 completed

83 Failed downloads:
['PYPL', 'NCLH', 'HWM', 'CFG', 'AMCR', 'HPE', 'CTLT', 'NWS', 'OTIS', 'APTV', 'UA', 'CARR', 'FANG', 'FOX', 'KEYS', 'HII', 'KHC', 'LB', 'ABBV', 'PAYC', 'LW', 'XYL', 'ZTS', 'ETSY', 'NOW', 'INFO', 'VNT', 'ANET', 'MPC', 'FTV', 'IR', 'CDW', 'ALLE', 'NWSA', 'QRVO', 'HCA', 'DOW', 'FISV', 'KMI', 'CTVA', 'HLT', 'PSX', 'SYF', 'IQV', 'FOXA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-01-01 -> 2011-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 946702800, endDate = 1293858000")')
['ANTM', 'FLT', 'PBCT', 'PEAK', 'CXO', 'ABMD', 'DISCK', 'NLOK', 'FRC', 'FB', 'VAR', 'FLIR', 'WLTW', 'ALXN', 'ATVI', 'COG', 'SIVB', 'BLL', 'NLSN', 'TIF', 'TWTR', 'ABC', 'DISCA', 'DRE', 'CERN', 'DISH', 'MXIM', 'CTXS', 'XLNX', 'WRK', 'PKI', 'VIAC', 'KSU', 'RE', 'GPS', 'HFC', 'FBHS', 'PXD']: YFTzMissingError('$%tic

In [14]:
train_df_smp, test_df_smp, scales_smp = generate_train_test_df_around_point(
    window=365 * 35, horizon=365, dfs=smp, for_prophet=True
)
train_df_ticker, test_df_tickers, scales_tickers = generate_train_test_df_around_point(
    window=365 * 1, horizon=365, dfs=smp + smp_tickers, for_prophet=True
)

In [15]:
context_prophet = Prophet(seasonality_mode="multiplicative")
context_prophet.fit(train_df_smp[0])
context_future = context_prophet.make_future_dataframe(
    periods=365, include_history=True
)
context_yhat = context_prophet.predict(context_future)
context_metrics = get_prophet_metrics(test_df_smp, [context_yhat], 365)
context_metrics

,mse,rmse,mae,mape
^GSPC,0.016975,0.130289,0.126827,0.181457


In [16]:
prophet_forecasts = []

for df, df_test in tqdm(zip(train_df_ticker, test_df_tickers)):
    prophet = Prophet(seasonality_mode="multiplicative")
    # prophet.add_regressor("smp_weekly", standardize=False, mode="additive")
    # prophet.add_regressor("smp_yearly", standardize=False, mode="additive")
    # prophet.add_regressor("smp_yhat", standardize=False, mode="additive")

    train_df = df.copy()
    train_df["smp_weekly"] = context_yhat[
        (context_yhat["ds"] >= train_df["ds"].iloc[0])
        & (context_yhat["ds"] <= train_df["ds"].iloc[-1])
    ]["weekly"].to_numpy()
    train_df["smp_yhat"] = context_yhat[
        (context_yhat["ds"] >= train_df["ds"].iloc[0])
        & (context_yhat["ds"] <= train_df["ds"].iloc[-1])
    ]["yhat"].to_numpy()
    train_df["smp_yearly"] = context_yhat[
        (context_yhat["ds"] >= train_df["ds"].iloc[0])
        & (context_yhat["ds"] <= train_df["ds"].iloc[-1])
    ]["yearly"].to_numpy()

    prophet.fit(train_df)

    future = prophet.make_future_dataframe(periods=365, include_history=True)
    future["smp_weekly"] = context_yhat[
        (context_yhat["ds"] >= future["ds"].iloc[0])
        & (context_yhat["ds"] <= future["ds"].iloc[-1])
    ]["weekly"].to_numpy()
    future["smp_yhat"] = context_yhat[
        (context_yhat["ds"] >= future["ds"].iloc[0])
        & (context_yhat["ds"] <= future["ds"].iloc[-1])
    ]["yhat"].to_numpy()
    future["smp_yearly"] = context_yhat[
        (context_yhat["ds"] >= future["ds"].iloc[0])
        & (context_yhat["ds"] <= future["ds"].iloc[-1])
    ]["yearly"].to_numpy()

    prophet_forecasts.append(prophet.predict(future))

prophet_metrics = get_prophet_metrics(test_df_tickers, prophet_forecasts, 365)
prophet_metrics["mape"].mean()

412it [01:07,  6.12it/s]


0.24158762029849162

In [17]:
prophet_metrics

,mse,rmse,mae,mape
^GSPC,0.021052,0.145092,0.108518,0.127334
VTR,0.101716,0.318930,0.273076,0.294286
LYV,0.022714,0.150713,0.116617,0.177962
TFX,0.009079,0.095284,0.076339,0.087686
PNR,0.003482,0.059005,0.048891,0.058709
...,...,...,...,...
IBM,0.069838,0.264270,0.220894,0.210641
SBUX,0.241795,0.491727,0.411727,0.330575
HIG,0.028214,0.167969,0.132016,0.363267
ETR,0.029872,0.172835,0.148824,0.197391


# Test model

In [14]:
def run_test(idx, point):
    train_df_smp, test_df_smp, scales_smp = generate_train_test_df_around_point(
        window=365 * 35, horizon=365, dfs=smp, for_prophet=False, point=point
    )
    model = final_models[idx]
    model.fit(train_df_smp, progressbar=False)
    map_approx = model.map_approx
    model_metrics = []

    for smp_ticker in tqdm(smp + smp_tickers):
        check = generate_train_test_df_around_point(
            window=365 * 1,
            horizon=365,
            dfs=[smp_ticker],
            for_prophet=False,
            point=point,
        )
        if check is None:
            continue

        train_df_tickers, test_df_tickers, scales_tickers = check
        model.map_approx = map_approx
        model.tune(train_df_tickers, progressbar=False)
        yhat = model.predict(365)
        yhat.to_csv(f"./out/model_{idx}_series_{smp_ticker['series'].iloc[0]}.csv")
        model_metrics.append(model.metrics(test_df_tickers, yhat, pool_cols="series"))

    print(f"{idx} - {model}: {pd.concat(model_metrics)['mape'].mean()}")
    return pd.concat(model_metrics)

In [15]:
# from joblib import Parallel, delayed


point = "2006-01-01"
all_metrics = []
# all_metrics = Parallel(n_jobs=8, prefer="threads")(
#     delayed(run_test)(idx, point) for idx, model in enumerate(final_models[:5])
# )

with open("./out/models.txt", "w") as f:
    for idx, model in enumerate(final_models):
        f.write(f"model_{idx}: {model}\n")

for idx, model in enumerate(final_models):
    all_metrics.append(run_test(idx, point))

100%|██████████| 504/504 [41:14<00:00,  4.91s/it]


0 - LT(n=25,r=0.8,at=False,complete) * (FS(p=365.25,n=10,at=True,complete) + FS(p=91.3125,n=7,at=True,complete) + FS(p=30.4375,n=4,at=True,complete) + FS(p=7,n=3,at=True,complete)): 0.15715462354219975


 55%|█████▍    | 277/504 [26:56<23:56,  6.33s/it] 

In [118]:
for metrics, model in zip(all_metrics, final_models):
    print(f"{model}: {metrics["mape"].mean()}")

LT(n=25,r=0.8,complete) * (FS(p=365.25,n=10,complete) + FS(p=91.3125,n=7,complete) + FS(p=30.4375,n=4,complete) + FS(p=7,n=3,complete)): 0.23633406794753192
LT(n=25,r=0.8,complete) * (FS(p=365.25,n=10,complete) + FS(p=91.3125,n=7,complete) + FS(p=30.4375,n=4,complete)): 0.22220762920917422
LT(n=25,r=0.8,complete) * (FS(p=365.25,n=10,complete) + FS(p=91.3125,n=7,complete) + FS(p=7,n=3,complete)): 0.23315341666383999
LT(n=25,r=0.8,complete) * (FS(p=365.25,n=10,complete) + FS(p=91.3125,n=7,complete)): 0.2319121997562418
LT(n=25,r=0.8,complete) * (FS(p=365.25,n=10,complete) + FS(p=91.3125,n=7,complete) + FS(p=30.4375,n=5,complete) + FS(p=7,n=3,complete)): 0.24844633535277494
LT(n=25,r=0.8,complete) * (FS(p=365.25,n=10,complete) + FS(p=91.3125,n=7,complete) + FS(p=30.4375,n=5,complete)): 0.21833104574490267
LT(n=25,r=0.8,complete) * (FS(p=365.25,n=10,complete) + FS(p=91.3125,n=7,complete) + FS(p=30.4375,n=6,complete) + FS(p=7,n=3,complete)): 0.24440633323110628
LT(n=25,r=0.8,complete) * (FS

In [106]:
all_metrics

,mse,rmse,mae,mape
AMZN,0.377487,0.614400,0.564832,0.413054
MAR,0.111828,0.334407,0.319255,0.312104
BIO,0.110366,0.332214,0.308669,0.342185
ADSK,0.194641,0.441182,0.413002,0.542215
MCO,0.083241,0.288515,0.263073,0.423926
NEE,0.006836,0.082679,0.069304,0.078973
MO,0.044383,0.210672,0.192508,0.177721
VMC,0.060941,0.246863,0.230722,0.369754


In [137]:
prophet_metrics.iloc[:9].mean()

mse     0.051284
rmse    0.204085
mae     0.169703
mape    0.204375
dtype: float64